In [2]:
from fastapi import APIRouter, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from typing import Optional
from passlib.context import CryptContext

In [ ]:
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

In [13]:
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
hashed_pwd = pwd_context.hash("secret")
print(hashed_pwd)

$2b$12$MKfV6atcq.rZcRS4XIYUmORjCg/Ym1CyeUWfDr9Blf9kWNNWmPNae


In [19]:
# User model
class User(BaseModel):
    username: str
    full_name: Optional[str] = None
    email: Optional[str] = None
    disabled: Optional[bool] = None
    
# User in database model
class UserInDB(User):
    hashed_password: str
    
# Fake database for demonstration purposes
fake_users_db = {
    "johndoe": UserInDB(
        username="johndoe",
        full_name="John Doe",
        email="johndoe@example.com",
        hashed_password=pwd_context.hash("secret")
    )
}
fake_users_db

{'johndoe': UserInDB(username='johndoe', full_name='John Doe', email='johndoe@example.com', disabled=None, hashed_password='$2b$12$r4nnx1zov6GMtH4LvcZCEeE6DTISn4BYfMrQud2e7f07YfL19GmPy')}

In [ ]:
def get_user(db, username: str) -> Optional[UserInDB]:
    if username in db:
        user_obj = db[username]
        if isinstance(user_obj, UserInDB):
            return user_obj
        return UserInDB(**user_obj)
    return None

get_user(fake_users_db, "johndoe")

UserInDB(username='johndoe', full_name='John Doe', email='johndoe@example.com', disabled=None, hashed_password='$2b$12$r4nnx1zov6GMtH4LvcZCEeE6DTISn4BYfMrQud2e7f07YfL19GmPy')

In [23]:
def verify_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)

verify_password("secret", "$2b$12$r4nnx1zov6GMtH4LvcZCEeE6DTISn4BYfMrQud2e7f07YfL19GmPy")

True

In [26]:
def authenticate_user(db, username: str, password: str) -> Optional[UserInDB]:
    user = get_user(db, username)
    if not user:
        return None
    if not verify_password(password, user.hashed_password):
        return None
    return user

authenticate_user(fake_users_db, "johndoe", "secret")

UserInDB(username='johndoe', full_name='John Doe', email='johndoe@example.com', disabled=None, hashed_password='$2b$12$r4nnx1zov6GMtH4LvcZCEeE6DTISn4BYfMrQud2e7f07YfL19GmPy')